In [1]:
import pandas as pd
import numpy as np
import glob
import os
from pathlib import Path
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

def merge_word_analogy_experiments(
    directory_path: str = "../Outer_Correlation/new_outer_correlation_results_per_section/",
    output_filename: str = "merged_word_analogy_results.csv",
    output_dir: str = ".",
    validate_data: bool = True,
    create_summary: bool = True
) -> Tuple[pd.DataFrame, Dict]:
    """
    Comprehensive merging of word analogy experiment results across all 14 categories.
    
    Designed specifically for word analogy research comparing similarity measures
    across semantic and syntactic relationship types.
    
    Args:
        directory_path: Path to directory containing the 14 CSV files
        output_filename: Name for the merged CSV file
        output_dir: Directory to save the merged file
        validate_data: Whether to perform data validation checks
        create_summary: Whether to create detailed summary statistics
        
    Returns:
        tuple: (merged_dataframe, summary_statistics)
    """
    
    print("Starting comprehensive merge of word analogy experiments...")
    print(f"Looking for CSV files in: {directory_path}")
    
    # Get all CSV files in the directory
    csv_files = glob.glob(os.path.join(directory_path, "*.csv"))
    
    if not csv_files:
        raise FileNotFoundError(f"No CSV files found in {directory_path}")
    
    print(f"Found {len(csv_files)} CSV files to merge")
    
    # Expected categories
    expected_categories = {
        'capital_common_countries_results.csv': 'capital-common-countries',
        'capital_world_results.csv': 'capital-world',
        'currency_results.csv': 'currency',
        'city_in_state_results.csv': 'city-in-state',
        'family_results.csv': 'family',
        'gram1_adjective_to_adverb_results.csv': 'gram1-adjective-to-adverb',
        'gram2_opposite_results.csv': 'gram2-opposite',
        'gram3_comparative_results.csv': 'gram3-comparative',
        'gram4_superlative_results.csv': 'gram4-superlative',
        'gram5_present_participle_results.csv': 'gram5-present-participle',
        'gram6_nationality_adjective_results.csv': 'gram6-nationality-adjective',
        'gram7_past_tense_results.csv': 'gram7-past-tense',
        'gram8_plural_results.csv': 'gram8-plural',
        'gram9_plural_verbs_results.csv': 'gram9-plural-verbs'
    }
    
    # Define semantic vs syntactic categories for analysis
    semantic_categories = {
        'capital-common-countries', 'capital-world', 'currency', 
        'city-in-state', 'family'
    }
    
    dataframes = []
    dataset_metadata = {}
    validation_issues = []
    
    # Process each CSV file
    for i, file_path in enumerate(sorted(csv_files)):
        filename = os.path.basename(file_path)
        dataset_name = filename.replace('.csv', '')
        
        print(f" Processing {filename}...")
        
        try:
            df = pd.read_csv(file_path)
            original_rows = len(df)
            
            # Add source tracking metadata
            df['dataset_source'] = dataset_name
            df['dataset_id'] = i
            df['file_path'] = file_path
            df['filename'] = filename
            
            # Validate expected category if we can map it
            expected_category = expected_categories.get(filename)
            if expected_category and 'category' in df.columns:
                actual_categories = df['category'].unique()
                if len(actual_categories) == 1 and actual_categories[0] != expected_category:
                    validation_issues.append(
                        f"Category mismatch in {filename}: expected {expected_category}, "
                        f"found {actual_categories[0]}"
                    )
            
            # Ensure category_type is properly set
            if 'category' in df.columns and 'category_type' in df.columns:
                df['category_type'] = df['category'].apply(
                    lambda x: 'semantic' if x in semantic_categories else 'syntactic'
                )
            
            # Store metadata about this dataset
            dataset_metadata[dataset_name] = {
                'filename': filename,
                'rows': original_rows,
                'categories': df['category'].unique().tolist() if 'category' in df.columns else [],
                'measures': df['measure'].unique().tolist() if 'measure' in df.columns else [],
                'quantiles': sorted(df['quantile'].unique().tolist()) if 'quantile' in df.columns else [],
                'rcond_values': sorted(df['rcond'].unique().tolist()) if 'rcond' in df.columns else [],
                'top_k_values': sorted(df['top@k'].unique().tolist()) if 'top@k' in df.columns else [],
                'accuracy_range': (
                    df['overall_accuracy'].min(), 
                    df['overall_accuracy'].max()
                ) if 'overall_accuracy' in df.columns else (None, None),
                'mean_accuracy': df['overall_accuracy'].mean() if 'overall_accuracy' in df.columns else None
            }
            
            dataframes.append(df)
            print(f"   Loaded {original_rows} rows")
            
        except Exception as e:
            error_msg = f"Error loading {filename}: {str(e)}"
            print(f"   {error_msg}")
            validation_issues.append(error_msg)
            continue
    
    if not dataframes:
        raise RuntimeError("No valid dataframes were loaded!")
    
    # Merge all dataframes
    print("\n Merging all datasets...")
    merged_df = pd.concat(dataframes, ignore_index=True)
    original_total_rows = len(merged_df)
    
    # Add computed features for analysis
    print(" Computing analysis features...")
    
    # Create parameter combination identifier
    if all(col in merged_df.columns for col in ['quantile', 'rcond', 'measure']):
        merged_df['param_combination'] = (
            merged_df['quantile'].astype(str) + '_' + 
            merged_df['rcond'].astype(str) + '_' + 
            merged_df['measure'].astype(str)
        )
    
    # Add percentile rankings within each dataset
    if 'overall_accuracy' in merged_df.columns and 'dataset_source' in merged_df.columns:
        merged_df['accuracy_percentile_within_dataset'] = merged_df.groupby('dataset_source')['overall_accuracy'].rank(pct=True)
        merged_df['accuracy_percentile_global'] = merged_df['overall_accuracy'].rank(pct=True)
        merged_df['is_top_10_percent_within_dataset'] = merged_df['accuracy_percentile_within_dataset'] >= 0.9
        merged_df['is_top_10_percent_global'] = merged_df['accuracy_percentile_global'] >= 0.9
    
    # Add category-level rankings
    if all(col in merged_df.columns for col in ['category', 'overall_accuracy']):
        merged_df['accuracy_rank_within_category'] = merged_df.groupby('category')['overall_accuracy'].rank(ascending=False)
        merged_df['is_best_in_category'] = merged_df['accuracy_rank_within_category'] == 1
    
    # Add measure performance indicators
    if all(col in merged_df.columns for col in ['measure', 'overall_accuracy']):
        merged_df['accuracy_rank_within_measure'] = merged_df.groupby('measure')['overall_accuracy'].rank(ascending=False)
    
    # Validation checks
    if validate_data:
        print("\n Performing data validation...")
        
        # Check for expected columns
        expected_columns = [
            'word1', 'word2', 'word3', 'true_word', 'category', 'category_type',
            'quantile', 'rcond', 'measure', 'top@k', 'overall_accuracy'
        ]
        missing_columns = [col for col in expected_columns if col not in merged_df.columns]
        if missing_columns:
            validation_issues.append(f"Missing expected columns: {missing_columns}")
        
        # Check for duplicate rows
        duplicate_rows = merged_df.duplicated().sum()
        if duplicate_rows > 0:
            validation_issues.append(f"Found {duplicate_rows} duplicate rows")
        
        # Check for missing values in key columns
        key_columns = ['category', 'measure', 'overall_accuracy']
        for col in key_columns:
            if col in merged_df.columns:
                missing_count = merged_df[col].isnull().sum()
                if missing_count > 0:
                    validation_issues.append(f"Missing values in {col}: {missing_count}")
        
        # Print validation results
        if validation_issues:
            print("  Validation Issues Found:")
            for issue in validation_issues:
                print(f"   - {issue}")
        else:
            print(" All validation checks passed!")
    
    # Create comprehensive summary statistics
    summary_stats = {
        'merge_info': {
            'total_files_processed': len(csv_files),
            'successful_files': len(dataframes),
            'total_rows': len(merged_df),
            'unique_categories': merged_df['category'].nunique() if 'category' in merged_df.columns else 0,
            'unique_measures': merged_df['measure'].nunique() if 'measure' in merged_df.columns else 0,
            'unique_param_combinations': merged_df['param_combination'].nunique() if 'param_combination' in merged_df.columns else 0,
        },
        'dataset_metadata': dataset_metadata,
        'validation_issues': validation_issues,
    }
    
    if create_summary and 'overall_accuracy' in merged_df.columns:
        print(" Creating detailed summary statistics...")
        
        # Overall performance summary
        summary_stats['performance_summary'] = {
            'overall_accuracy_range': (merged_df['overall_accuracy'].min(), merged_df['overall_accuracy'].max()),
            'overall_accuracy_mean': merged_df['overall_accuracy'].mean(),
            'overall_accuracy_std': merged_df['overall_accuracy'].std(),
        }
        
        # Performance by category
        if 'category' in merged_df.columns:
            category_stats = merged_df.groupby('category')['overall_accuracy'].agg([
                'count', 'mean', 'std', 'min', 'max'
            ]).round(4)
            summary_stats['category_performance'] = category_stats.to_dict('index')
        
        # Performance by measure
        if 'measure' in merged_df.columns:
            measure_stats = merged_df.groupby('measure')['overall_accuracy'].agg([
                'count', 'mean', 'std', 'min', 'max'
            ]).round(4)
            summary_stats['measure_performance'] = measure_stats.to_dict('index')
        
        # Performance by category type
        if 'category_type' in merged_df.columns:
            category_type_stats = merged_df.groupby('category_type')['overall_accuracy'].agg([
                'count', 'mean', 'std', 'min', 'max'
            ]).round(4)
            summary_stats['category_type_performance'] = category_type_stats.to_dict('index')
        
        # Best parameter combinations
        if 'param_combination' in merged_df.columns:
            best_params = merged_df.nlargest(20, 'overall_accuracy')[
                ['param_combination', 'category', 'measure', 'quantile', 'rcond', 'overall_accuracy']
            ]
            summary_stats['top_parameter_combinations'] = best_params.to_dict('records')
    
    # Save the merged dataset
    output_path = Path(output_dir) / output_filename
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    print(f"\n Saving merged dataset to: {output_path}")
    merged_df.to_csv(output_path, index=False)
    
    # Also save summary statistics as JSON
    summary_path = Path(output_dir) / f"summary_{output_filename.replace('.csv', '.json')}"
    import json
    
    # Convert numpy types to Python types for JSON serialization
    def convert_numpy_types(obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return obj
    
    # Clean summary stats for JSON serialization
    json_summary = {}
    for key, value in summary_stats.items():
        if isinstance(value, dict):
            json_summary[key] = {k: convert_numpy_types(v) for k, v in value.items()}
        else:
            json_summary[key] = convert_numpy_types(value)
    
    with open(summary_path, 'w') as f:
        json.dump(json_summary, f, indent=2, default=str)
    
    print(f" Summary statistics saved to: {summary_path}")
    
    # Print final summary
    print(f"\n Merge completed successfully!")
    print(f" Final dataset: {len(merged_df):,} rows, {len(merged_df.columns)} columns")
    print(f"  Categories: {merged_df['category'].nunique() if 'category' in merged_df.columns else 'N/A'}")
    print(f" Measures: {merged_df['measure'].nunique() if 'measure' in merged_df.columns else 'N/A'}")
    print(f" Accuracy range: {merged_df['overall_accuracy'].min():.4f} - {merged_df['overall_accuracy'].max():.4f}")
    
    return merged_df, summary_stats

In [2]:
# Quick usage function
def quick_merge_data():
    """Quick function to merge word analogy data with sensible defaults"""
    return merge_word_analogy_experiments(
        directory_path="../Outer_Correlation/new_outer_correlation_results_per_section/",
        output_filename="oc_results.csv",
        output_dir="../Outer_Correlation/final_results/",
        validate_data=True,
        create_summary=True
    )

In [3]:
merged_data, summary = quick_merge_data()

Starting comprehensive merge of word analogy experiments...
Looking for CSV files in: ../Outer_Correlation/new_outer_correlation_results_per_section/
Found 14 CSV files to merge
 Processing capital_common_countries_results.csv...
   Loaded 30360 rows
 Processing capital_world_results.csv...
   Loaded 71556 rows
 Processing city_in_state_results.csv...
   Loaded 112488 rows
 Processing currency_results.csv...
   Loaded 7680 rows
 Processing family_results.csv...
   Loaded 20520 rows
 Processing gram1_adjective_to_adverb_results.csv...
   Loaded 48720 rows
 Processing gram2_opposite_results.csv...
   Loaded 22800 rows
 Processing gram3_comparative_results.csv...
   Loaded 79920 rows
 Processing gram4_superlative_results.csv...
   Loaded 27144 rows
 Processing gram5_present_participle_results.csv...
   Loaded 52200 rows
 Processing gram6_nationality_adjective_results.csv...
   Loaded 73740 rows
 Processing gram7_past_tense_results.csv...
   Loaded 88920 rows
 Processing gram8_plural_resul

In [4]:
print("Dataset overview:")
print(f"Shape: {merged_data.shape}")
print(f"Categories: {sorted(merged_data['category'].unique())}")
print(f"Measures: {sorted(merged_data['measure'].unique())}")
print(f"Quantiles tested: {sorted(merged_data['quantile'].unique())}")

Dataset overview:
Shape: (737688, 34)
Categories: ['capital-common-countries', 'capital-world', 'city-in-state', 'currency', 'family', 'gram1-adjective-to-adverb', 'gram2-opposite', 'gram3-comparative', 'gram4-superlative', 'gram5-present-participle', 'gram6-nationality-adjective', 'gram7-past-tense', 'gram8-plural', 'gram9-plural-verbs']
Measures: ['mahalanobis_cosine', 'mahalanobis_shifted_cosine', 'naive_cosine']
Quantiles tested: [0.01, 0.05, 0.1, 0.25, 0.5]
